In [1]:
import os
import time
import requests
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse

In [2]:
df = pd.read_json("./data/items.json")

Get GIF urls

In [3]:
df['gif_url'] = df.image_url.str[0]

In [4]:
gif_df = df[~pd.isnull(df.gif_url)][['id', 'gif_url']].copy()

In [5]:
assert gif_df.notna().all().all()

Get JPG urls

In [19]:
df['jpg_url'] = df.image_url.str[-1]

In [21]:
jpg_df = df[~pd.isnull(df.jpg_url)][['id', 'jpg_url']].copy()

In [22]:
assert jpg_df.notna().all().all()

Download urls

In [23]:
def download_url(url, target_dir="img/gif", verbose=False):
    parsed_url = urlparse(url)
    basename = os.path.basename(parsed_url.path)
    filepath = os.path.join(target_dir, basename)
    if not os.path.exists(filepath):
        if verbose:
            print(f"Downloading {url}")
        r = requests.get(url)
        if not r.ok:
            if r.status_code == 404:
                print(f"404: {url}")
            else:
                raise ValueError(f"Error code: {r.status_code} Check: {url}")
        with open(filepath, 'wb') as f:
            f.write(r.content)
        time.sleep(0.1)
    return filepath

In [24]:
len(gif_df)

35035

In [32]:
tqdm.pandas(desc="Download GIFs")

In [33]:
gif_df['gif_path'] = gif_df.gif_url.progress_apply(download_url)

Download GIFs: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35035/35035 [1:18:12<00:00,  7.47it/s]


In [28]:
# tqdm.pandas(desc="Download JPGs")

In [29]:
# jpg_df['jpg_path'] = jpg_df.jpg_url.progress_apply(lambda x: download_url(x, target_dir="img/jpg"))

Download JPGs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35035/35035 [00:00<00:00, 74119.37it/s]
